
### Reference:

- Paper : Temporal Fusion Transformers for Interpretable Multi-horizon Time Series Forecasting (https://arxiv.org/abs/1912.09363)
- Keras Docs https://keras.io/examples/structured_data/classification_with_grn_and_vsn/


In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [1]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    batch_size = strategy.num_replicas_in_sync * 64
    print("Running on TPU:", tpu.master())
    print(f"Batch Size: {batch_size}")
    
except ValueError:
    strategy = tf.distribute.get_strategy()
    batch_size = 256
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    print(f"Batch Size: {batch_size}")

In [ ]:
file_path = '/kaggle/input/tabular-playground-series-nov-2021/'

train = pd.read_csv(file_path + 'train.csv')
test = pd.read_csv(file_path + 'test.csv')
sub = pd.read_csv(file_path + 'sample_submission.csv')

In [ ]:
print(f'Train data shape: {train.shape}')
print(f'Test data shape: {test.shape}')

In [ ]:
feature_names = [f'f{i}' for i in range(100)]
csv_header = feature_names + ['target']
target = train['target']

In [ ]:
# feature scaling
scaler = StandardScaler()
train = scaler.fit_transform(train[feature_names])
test = scaler.transform(test[feature_names])

scaler = MinMaxScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

train = pd.DataFrame(train, columns=feature_names)
test = pd.DataFrame(test, columns=feature_names)
train['target'] = target
test['target'] = target

In [ ]:
random_selection = np.random.rand(len(train.index)) <= 0.85
train_data = train[random_selection]
valid_data = train[~random_selection]
test_data = test.copy()

train_data.to_csv('train_data.csv', index=False, header=False)
valid_data.to_csv('valid_data.csv', index=False, header=False)
test_data.to_csv('test_data.csv', index=False, header=False)

print(f'Train data shape: {train_data.shape}')
print(f'Valid data shape: {valid_data.shape}')
print(f'Test data shape: {test.shape}')

In [ ]:
def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128):
    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=csv_header,
        label_name='target',
        num_epochs=1,
        header=False,
        shuffle=shuffle,
    )
    return dataset

In [ ]:
def create_model_inputs():
    inputs = {}
    for feature_name in feature_names:
        inputs[feature_name] = layers.Input(name=feature_name, shape=(), dtype=tf.float32)
    return inputs

In [ ]:
def encode_inputs(inputs, encoding_size):
    encoded_features = []
    for feature_name in inputs:
        # Project the numeric feature to encoding_size using linear transformation.
        encoded_feature = tf.expand_dims(inputs[feature_name], -1)
        encoded_feature = layers.Dense(units=encoding_size)(encoded_feature)
        encoded_features.append(encoded_feature)
    return encoded_features

In [ ]:
class GatedLinearUnit(layers.Layer):
    def __init__(self, units):
        super(GatedLinearUnit, self).__init__()
        self.linear = layers.Dense(units)
        self.sigmoid = layers.Dense(units, activation="sigmoid")

    def call(self, inputs):
        return self.linear(inputs) * self.sigmoid(inputs)

In [ ]:
class GatedResidualNetwork(layers.Layer):
    def __init__(self, units, dropout_rate):
        super(GatedResidualNetwork, self).__init__()
        self.units = units
        self.elu_dense = layers.Dense(units, activation="elu")
        self.linear_dense = layers.Dense(units)
        self.dropout = layers.Dropout(dropout_rate)
        self.gated_linear_unit = GatedLinearUnit(units)
        self.layer_norm = layers.LayerNormalization()
        self.project = layers.Dense(units)

    def call(self, inputs):
        x = self.elu_dense(inputs)
        x = self.linear_dense(x)
        x = self.dropout(x)
        if inputs.shape[-1] != self.units:
            inputs = self.project(inputs)
        x = inputs + self.gated_linear_unit(x)
        x = self.layer_norm(x)
        return x

In [ ]:
class VariableSelection(layers.Layer):
    def __init__(self, num_features, units, dropout_rate):
        super(VariableSelection, self).__init__()
        self.grns = list()
        # Create a GRN for each feature independently
        for idx in range(num_features):
            grn = GatedResidualNetwork(units, dropout_rate)
            self.grns.append(grn)
        # Create a GRN for the concatenation of all the features
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = layers.Dense(units=num_features, activation="softmax")

    def call(self, inputs):
        v = layers.concatenate(inputs)
        v = self.grn_concat(v)
        v = tf.expand_dims(self.softmax(v), axis=-1)

        x = []
        for idx, inp in enumerate(inputs):
            x.append(self.grns[idx](inp))
        x = tf.stack(x, axis=1)

        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs

In [ ]:
def create_model(encoding_size):
    inputs = create_model_inputs()
    feature_list = encode_inputs(inputs, encoding_size)
    num_features = len(feature_list)

    features = VariableSelection(num_features, encoding_size, dropout_rate)(feature_list)

    outputs = layers.Dense(units=1, activation="sigmoid")(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
learning_rate = 0.001
dropout_rate = 0.15
batch_size = 256
num_epochs = 20
encoding_size = 256

with strategy.scope():
    
    model = create_model(encoding_size)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[keras.metrics.AUC()],
    )

    es = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=20, restore_best_weights=True
    )

    pl = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.1, patience=20, verbose=0,
        mode='auto', min_delta=0.0001, cooldown=0, min_lr=0,
    )
    

    print("Start training the model ...")
    train_dataset = get_dataset_from_csv('train_data.csv', shuffle=True, batch_size=batch_size)
    valid_dataset = get_dataset_from_csv('valid_data.csv', batch_size=batch_size)
    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        callbacks=[es, pl],
    )
    print("Model training Completed.")

    
    print("Predicting model performance ...")
    test_dataset = get_dataset_from_csv('test_data.csv', batch_size=batch_size)
    preds = model.predict(test_dataset)
    print("Prediction Completed.")

In [ ]:
sub['target'] = preds
sub.to_csv('submission.csv', index=False)